# KoBERT finetuning

## 특허문서의 title, abstract, claim과 산업분류 코드를 활용하여 IPC를 예측

In [1]:
!pip install gluonnlp
# !pip install mxnet
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-xsfmfifg/kobert-tokenizer_27d667ec126240eab429da1a3505506c
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-xsfmfifg/kobert-tokenizer_27d667ec126240eab429da1a3505506c
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-i5c4ykjy
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-i5c4ykjy


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_excel('/content/drive/MyDrive/patent_data/활용특허데이터.xlsx')

In [5]:
def split_ipc(text:str):
  return text.split('-')[0]

df['target'] = df['메인 IPC'].map(split_ipc)

def split_category(text:str):
  return text.split('_')[1]

df['category'] = df['세세분류'].map(split_category)

In [7]:
df[['메인 IPC', 'target']].head()

,메인 IPC,target
0,G06F-001/32,G06F
1,G06F-013/20,G06F
2,G06F-011/07,G06F
3,H04L-009/32,H04L
4,G06F-011/08,G06F


In [8]:
df[['세세분류', 'category']].head()

,세세분류,category
0,62090_기타 정보 기술 및 컴퓨터 운영 관련 서비스업,기타 정보 기술 및 컴퓨터 운영 관련 서비스업
1,62090_기타 정보 기술 및 컴퓨터 운영 관련 서비스업,기타 정보 기술 및 컴퓨터 운영 관련 서비스업
2,62090_기타 정보 기술 및 컴퓨터 운영 관련 서비스업,기타 정보 기술 및 컴퓨터 운영 관련 서비스업
3,62090_기타 정보 기술 및 컴퓨터 운영 관련 서비스업,기타 정보 기술 및 컴퓨터 운영 관련 서비스업
4,62090_기타 정보 기술 및 컴퓨터 운영 관련 서비스업,기타 정보 기술 및 컴퓨터 운영 관련 서비스업


In [9]:
feature = ['category', '발명의 명칭', '요약', '독립항']

In [10]:
df['text'] = df[feature[0]] + df[feature[1]] + df[feature[2]] + df[feature[3]]

In [11]:
df['text'].loc[0]

'기타 정보 기술 및 컴퓨터 운영 관련 서비스업운영 상태에 따른 컴퓨터 본체 전원 제어장치(computer main power supply control apparatus according to the operational condition)컴퓨터의 운영 상태별로 달라지는 소비전력 소모 상태를 감지하고, 이를 기반으로 컴퓨터 운영 상태에 따라 대기전원을 최적으로 제어하여 절전을 구현하며, 메인보드가 변경되어도 운영 상태별로 변경된 메인보드에 최적화되어 소비전력 측정 및 절전을 수행하여 범용성을 향상시킨 운영 상태에 따른 컴퓨터 본체 전원 제어장치에 관한 것으로서, 복수의 운영 모드로 컴퓨터를 운영하는 컴퓨터 메인보드, 및 컴퓨터 메인보드의 운영 모드 및 장착된 장치에 따라 달라지는 소비전력을 실시간으로 감지하여 대기전원이 불필요한 운영 상태이면 컴퓨터 메인보드에 공급되는 대기전원을 차단하여 전력 소모를 방지하고, 상기 컴퓨터 메인보드의 상태 변경시 이전 상태정보는 초기화하고 변경된 컴퓨터 메인보드에 따라 달라지는 소비전력을 실시간으로 감지하여 대기전원이 불필요한 운영 상태이면 변경된 컴퓨터 메인보드에 공급되는 대기전원을 차단하여 범용성을 유지하는 전원 공급장치를 포함하여, 운영 상태에 따른 컴퓨터 본체 전원 제어장치를 구현한다.[청구항1]\n복수의 운영 모드로 컴퓨터를 운영하는 컴퓨터 메인보드; 및\n상기 컴퓨터 메인보드의 운영 모드 및 장착된 장치에 따라 달라지는 소비전력을 실시간으로 감지하여 대기전원이 불필요한 운영 상태이면 컴퓨터 메인보드에 공급되는 대기전원을 차단하여 전력 소모를 방지하고, 상기 컴퓨터 메인보드의 상태 변경시 이전 상태정보는 초기화하고 변경된 컴퓨터 메인보드에 따라 달라지는 소비전력을 실시간으로 감지하여 대기전원이 불필요한 운영 상태이면 변경된 컴퓨터 메인보드에 공급되는 대기전원을 차단하여 범용성을 유지하는 전원 공급장치를 포함하고,\n상기 전원 공급장치는 컴퓨터 메인보드의 동작 모드가 상용전원 차단 상태, 오프 모드, 최대 절전 모드,

In [12]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [13]:
def text_to_token(df:pd.DataFrame):
  input_ids = list()
  token_type_ids = list()
  attention_mask = list()

  for i in range(len(df)):
    temp = tokenizer.encode_plus(df['text'].loc[i], add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask = True, 
            return_token_type_ids=True,
            truncation=True)
    input_ids.append(temp['input_ids'])
    token_type_ids.append(temp['token_type_ids'])
    attention_mask.append(temp['attention_mask'])

  return input_ids, token_type_ids, attention_mask

df['input_ids'], df['token_type_ids'], df['attention_mask'] = text_to_token(df)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
df[['input_ids', 'token_type_ids', 'attention_mask']].head()

,input_ids,token_type_ids,attention_mask
0,"[2, 1306, 4103, 1289, 2184, 4660, 3517, 1083, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[2, 1306, 4103, 1289, 2184, 4660, 3517, 1083, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[2, 1306, 4103, 1289, 2184, 4660, 3517, 1083, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[2, 1306, 4103, 1289, 2184, 4660, 3517, 1083, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[2, 1306, 4103, 1289, 2184, 4660, 3517, 1083, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
# def text_to_token(text:str):
#   token = list()
#   temp = tokenizer.encode(text)

#   return temp

# df['token'] = df['text'].map(text_to_token)

In [15]:
df.head(1).T

,0
No.,2392
중분류 코드,62
소분류 코드,620
세세분류 코드,62090
세세분류,62090_기타 정보 기술 및 컴퓨터 운영 관련 서비스업
국가코드,KR
법적상태,등록
발명의 명칭,운영 상태에 따른 컴퓨터 본체 전원 제어장치(computer main power s...
요약,"컴퓨터의 운영 상태별로 달라지는 소비전력 소모 상태를 감지하고, 이를 기반으로 컴퓨..."
독립항,[청구항1]\n복수의 운영 모드로 컴퓨터를 운영하는 컴퓨터 메인보드; 및\n상기 컴...


In [16]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [17]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [18]:
my_dict = {}
for i in range(len(df['target'].unique())):
  my_dict[df['target'].unique()[i]] = i

In [19]:
my_dict

{'A01B': 233,
 'A01C': 309,
 'A01D': 190,
 'A01F': 518,
 'A01G': 79,
 'A01H': 514,
 'A01J': 308,
 'A01K': 70,
 'A01M': 315,
 'A01N': 434,
 'A21C': 289,
 'A21D': 463,
 'A22B': 517,
 'A22C': 270,
 'A23B': 510,
 'A23C': 291,
 'A23D': 455,
 'A23F': 511,
 'A23G': 290,
 'A23J': 515,
 'A23K': 448,
 'A23L': 285,
 'A23N': 287,
 'A23P': 286,
 'A24B': 507,
 'A24C': 284,
 'A24D': 160,
 'A24F': 162,
 'A41B': 476,
 'A41C': 496,
 'A41D': 493,
 'A41F': 442,
 'A41G': 170,
 'A41H': 495,
 'A42B': 437,
 'A42C': 494,
 'A43B': 176,
 'A43C': 487,
 'A43D': 438,
 'A44B': 44,
 'A44C': 168,
 'A45B': 156,
 'A45C': 157,
 'A45D': 154,
 'A45F': 491,
 'A46B': 159,
 'A47B': 185,
 'A47C': 183,
 'A47F': 27,
 'A47G': 143,
 'A47H': 503,
 'A47J': 288,
 'A47K': 406,
 'A47L': 42,
 'A61B': 116,
 'A61C': 389,
 'A61D': 390,
 'A61F': 388,
 'A61G': 186,
 'A61H': 436,
 'A61J': 318,
 'A61K': 439,
 'A61L': 119,
 'A61M': 314,
 'A61N': 387,
 'A62B': 91,
 'A62C': 60,
 'A62D': 449,
 'A63B': 88,
 'A63C': 86,
 'A63D': 174,
 'A63F': 26,
 '

In [20]:
def target_to_label(text:str):
  return my_dict[text]

df['target_label'] = df['target'].map(target_to_label)

In [21]:
df[['target', 'target_label']].head()

,target,target_label
0,G06F,0
1,G06F,0
2,G06F,0
3,H04L,1
4,G06F,0


In [22]:
df.head(3).T

,0,1,2
No.,2392,2573,4033
중분류 코드,62,62,62
소분류 코드,620,620,620
세세분류 코드,62090,62090,62090
세세분류,62090_기타 정보 기술 및 컴퓨터 운영 관련 서비스업,62090_기타 정보 기술 및 컴퓨터 운영 관련 서비스업,62090_기타 정보 기술 및 컴퓨터 운영 관련 서비스업
국가코드,KR,KR,KR
법적상태,등록,등록,취하
발명의 명칭,운영 상태에 따른 컴퓨터 본체 전원 제어장치(computer main power s...,한대의본체로 다수가 사용하는 컴퓨터장치 및 그 운영방법(computer system...,자동 컴퓨터 시스템 장애 복구와 예방에 대한 방법 및 그 시스템(method for...
요약,"컴퓨터의 운영 상태별로 달라지는 소비전력 소모 상태를 감지하고, 이를 기반으로 컴퓨...","본 발명은 한대의본체로 다수가사용하는 컴퓨터장치 및 그 운영방법에 있어서, 한대의본...","본 발명의 특징에 따르면, 본 발명은 사용자의 운용이 이루어지는 사용자 컴퓨팅 시스..."
독립항,[청구항1]\n복수의 운영 모드로 컴퓨터를 운영하는 컴퓨터 메인보드; 및\n상기 컴...,"[청구항1]\n한대의본체로 다수가사용하는 컴퓨터장치에 있어서,\n다수의사용자가 동시...","[청구항1]\n사용자의 컴퓨터 시스템에서 운영체제가 구동되는 동안, 컴퓨터 시스템의..."


In [23]:
df['text'].loc[0]

'기타 정보 기술 및 컴퓨터 운영 관련 서비스업운영 상태에 따른 컴퓨터 본체 전원 제어장치(computer main power supply control apparatus according to the operational condition)컴퓨터의 운영 상태별로 달라지는 소비전력 소모 상태를 감지하고, 이를 기반으로 컴퓨터 운영 상태에 따라 대기전원을 최적으로 제어하여 절전을 구현하며, 메인보드가 변경되어도 운영 상태별로 변경된 메인보드에 최적화되어 소비전력 측정 및 절전을 수행하여 범용성을 향상시킨 운영 상태에 따른 컴퓨터 본체 전원 제어장치에 관한 것으로서, 복수의 운영 모드로 컴퓨터를 운영하는 컴퓨터 메인보드, 및 컴퓨터 메인보드의 운영 모드 및 장착된 장치에 따라 달라지는 소비전력을 실시간으로 감지하여 대기전원이 불필요한 운영 상태이면 컴퓨터 메인보드에 공급되는 대기전원을 차단하여 전력 소모를 방지하고, 상기 컴퓨터 메인보드의 상태 변경시 이전 상태정보는 초기화하고 변경된 컴퓨터 메인보드에 따라 달라지는 소비전력을 실시간으로 감지하여 대기전원이 불필요한 운영 상태이면 변경된 컴퓨터 메인보드에 공급되는 대기전원을 차단하여 범용성을 유지하는 전원 공급장치를 포함하여, 운영 상태에 따른 컴퓨터 본체 전원 제어장치를 구현한다.[청구항1]\n복수의 운영 모드로 컴퓨터를 운영하는 컴퓨터 메인보드; 및\n상기 컴퓨터 메인보드의 운영 모드 및 장착된 장치에 따라 달라지는 소비전력을 실시간으로 감지하여 대기전원이 불필요한 운영 상태이면 컴퓨터 메인보드에 공급되는 대기전원을 차단하여 전력 소모를 방지하고, 상기 컴퓨터 메인보드의 상태 변경시 이전 상태정보는 초기화하고 변경된 컴퓨터 메인보드에 따라 달라지는 소비전력을 실시간으로 감지하여 대기전원이 불필요한 운영 상태이면 변경된 컴퓨터 메인보드에 공급되는 대기전원을 차단하여 범용성을 유지하는 전원 공급장치를 포함하고,\n상기 전원 공급장치는 컴퓨터 메인보드의 동작 모드가 상용전원 차단 상태, 오프 모드, 최대 절전 모드,

In [24]:
def remove_enter(text:str):
  return text.replace('\n',"")

df['text'] = df['text'].map(remove_enter)

In [25]:
df['text'].loc[0]

'기타 정보 기술 및 컴퓨터 운영 관련 서비스업운영 상태에 따른 컴퓨터 본체 전원 제어장치(computer main power supply control apparatus according to the operational condition)컴퓨터의 운영 상태별로 달라지는 소비전력 소모 상태를 감지하고, 이를 기반으로 컴퓨터 운영 상태에 따라 대기전원을 최적으로 제어하여 절전을 구현하며, 메인보드가 변경되어도 운영 상태별로 변경된 메인보드에 최적화되어 소비전력 측정 및 절전을 수행하여 범용성을 향상시킨 운영 상태에 따른 컴퓨터 본체 전원 제어장치에 관한 것으로서, 복수의 운영 모드로 컴퓨터를 운영하는 컴퓨터 메인보드, 및 컴퓨터 메인보드의 운영 모드 및 장착된 장치에 따라 달라지는 소비전력을 실시간으로 감지하여 대기전원이 불필요한 운영 상태이면 컴퓨터 메인보드에 공급되는 대기전원을 차단하여 전력 소모를 방지하고, 상기 컴퓨터 메인보드의 상태 변경시 이전 상태정보는 초기화하고 변경된 컴퓨터 메인보드에 따라 달라지는 소비전력을 실시간으로 감지하여 대기전원이 불필요한 운영 상태이면 변경된 컴퓨터 메인보드에 공급되는 대기전원을 차단하여 범용성을 유지하는 전원 공급장치를 포함하여, 운영 상태에 따른 컴퓨터 본체 전원 제어장치를 구현한다.[청구항1]복수의 운영 모드로 컴퓨터를 운영하는 컴퓨터 메인보드; 및상기 컴퓨터 메인보드의 운영 모드 및 장착된 장치에 따라 달라지는 소비전력을 실시간으로 감지하여 대기전원이 불필요한 운영 상태이면 컴퓨터 메인보드에 공급되는 대기전원을 차단하여 전력 소모를 방지하고, 상기 컴퓨터 메인보드의 상태 변경시 이전 상태정보는 초기화하고 변경된 컴퓨터 메인보드에 따라 달라지는 소비전력을 실시간으로 감지하여 대기전원이 불필요한 운영 상태이면 변경된 컴퓨터 메인보드에 공급되는 대기전원을 차단하여 범용성을 유지하는 전원 공급장치를 포함하고,상기 전원 공급장치는 컴퓨터 메인보드의 동작 모드가 상용전원 차단 상태, 오프 모드, 최대 절전 모드, 대기 모드

In [26]:
# dataset_train = df[['No.', 'text', 'target_label']][:int(len(df)*0.8)]
# dataset_test = df[['No.', 'text', 'target_label']][int(len(df)*0.8):]
dataset_train, dataset_test = train_test_split(df[['No.', 'text', 'target_label']], test_size=0.2, random_state=42, shuffle=True)
dataset_train.to_csv('dataset_train.txt', index=False, header=True, sep='\t', encoding='utf8')
dataset_test.to_csv('dataset_test.txt', index=False, header=True, sep='\t', encoding='utf8')

In [27]:
dataset_train = nlp.data.TSVDataset('/content/dataset_train.txt', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/dataset_test.txt', field_indices=[1,2], num_discard_samples=1)

In [28]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [29]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [30]:
## Setting parameters
max_len = 256
batch_size = 8
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [31]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [32]:
next(iter(data_train))

(array([   2, 2044, 2267, 7202, 6873, 6241, 6579, 7589,  743, 3036, 7565,
        4151, 2270,  522,  517,  398,  368,  438,  382,  377,  410,  517,
         423,  397,  403,  388,  706,  517,  440,  408,  415,  705,  367,
         432,  368,  421,  707,  517,  455,  376,  401,  410,  517,   40,
        2408, 2235, 6208, 2135, 6579, 7589,  743, 3036, 7565, 4151, 2270,
        6896, 1094,  909, 6553,  517,   46, 3945, 7088, 4465, 7815, 3533,
        6500, 4249, 7794, 1586,  517,  252, 3533, 6500, 3726, 7815, 3533,
        5788, 7088, 4128, 7342, 7794, 1586,  517,  252, 3533, 5788, 7088,
        2135, 6896, 1468, 3996, 6733, 1577,  517,   46,  517, 7233, 7086,
        3533, 5788, 7088,  881, 7253, 6720, 2872, 7463, 6723,  517,   46,
        2872, 7463, 5899, 3533, 5788, 7088, 3758, 7178, 6725, 5176, 5330,
        5452, 1052, 7088, 3803, 5152, 3704, 2909, 7794, 1586,  517,  252,
        5176, 5330, 5452, 1052, 7088,  879, 3533, 5788, 6896, 3358, 7088,
         517, 5330, 7815,  993, 7394, 

In [33]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [34]:
len(df['target_label'].unique())

520

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=520,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    # torch.cuda.empty_cache()

  0%|          | 0/697 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 6.302846908569336 train acc 0.0
epoch 1 batch id 201 loss 6.407012462615967 train acc 0.003109452736318408
epoch 1 batch id 401 loss 5.897708892822266 train acc 0.015897755610972567
epoch 1 batch id 601 loss 5.950346946716309 train acc 0.03348585690515807
epoch 1 train acc 0.036585365853658534


  0%|          | 0/175 [00:00<?, ?it/s]

epoch 1 test acc 0.07642857142857143


  0%|          | 0/697 [00:00<?, ?it/s]

...

epoch 19 test acc 0.5742857142857143


  0%|          | 0/697 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.31044405698776245 train acc 1.0
epoch 20 batch id 201 loss 1.138004183769226 train acc 0.9017412935323383
epoch 20 batch id 401 loss 0.5881264209747314 train acc 0.9089775561097256
epoch 20 batch id 601 loss 0.619038462638855 train acc 0.9103577371048253
epoch 20 train acc 0.9137374461979914


  0%|          | 0/175 [00:00<?, ?it/s]

epoch 20 test acc 0.5742857142857143
